# Unsupervised Learning: Types of Unsupervised Learning

Unsupervised learning is all about discovering hidden structure in unlabeled data. In this notebook, we'll explore several different clustering approaches and see how each one handles different shapes and patterns in data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, BisectingKMeans
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.metrics import silhouette_score, rand_score
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd
import seaborn as sns

## What is Unsupervised Learning?

Unsupervised learning deals with data that has **no labels**, meaning the algorithm must discover structure on its own. Instead of learning by example (like supervised learning), unsupervised methods identify patterns, groupings, or relationships based only on the data itself.

### Supervised vs Unsupervised: A Quick Recap
- **Supervised Learning:** You have inputs *and* outputs — like a study guide with an answer key. The model learns to map features to known labels.
- **Unsupervised Learning:** You only have inputs — like taking notes without knowing what the exam questions will be. The model looks for structure or grouping on its own.

### Why "Unsupervised"?
There’s no teacher, no correct answers provided ahead of time. The algorithm must uncover meaningful clusters or patterns entirely from the data structure.

### Common Real-World Uses
- 🎵 **Spotify:** Groups similar songs into playlists
- 🛒 **Retail:** Identifying customer segments for marketing
- 🧬 **Biology:** Grouping gene expression profiles
- 📰 **News:** Clustering articles by topics
- 🎮 **Gaming:** Detecting different play styles

### Intuition Example
Imagine sorting students into teams for an event without any categories given. You might group by behavior, interests, or who hangs out together — that's clustering!

## Centroid-Based Clustering

Centroid-based clustering groups data points according to their similarity to a central representative called a **centroid**. This centroid acts as the "center of mass" for the cluster.

### The Big Idea
Suppose you're organizing a pizza party. You want to place pizza stations (centroids) in a way that minimizes how far students must walk. Students naturally go to the station closest to them. Over time, stations would shift until each one is in the ideal location.

Clustering works similarly:
- Pick the number of clusters (number of pizza stations)
- Assign each data point to its nearest centroid
- Adjust centroids based on cluster members
- Repeat until stable

### Key Goals
- **Minimize distances within clusters** (points in the same cluster should be close)
- **Maximize distances between clusters** (clusters should be well-separated)

This makes centroid-based clustering intuitive and computationally efficient.

### Visualization of the Clustering Process

To get an intuition for how centroid-based methods work, let's walk through the process visually using K-Means as an example.

In [ ]:
# Create simple dataset
np.random.seed(42)
X_demo, _ = make_blobs(n_samples=150, centers=3, cluster_std=0.6, random_state=42)

# Manually show clustering steps
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

# Step 1: Raw data
axes[0].scatter(X_demo[:, 0], X_demo[:, 1], c='gray', s=80, alpha=0.6, edgecolors='black')
axes[0].set_title('Step 1: Raw Data (No Labels!)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].grid(True, alpha=0.3)

# Step 2: Random initial centroids
initial_centroids = X_demo[np.random.choice(len(X_demo), 3, replace=False)]
axes[1].scatter(X_demo[:, 0], X_demo[:, 1], c='gray', s=80, alpha=0.6, edgecolors='black')
axes[1].scatter(initial_centroids[:, 0], initial_centroids[:, 1], 
               c='red', s=400, marker='*', edgecolors='black', linewidth=2, label='Initial Centroids')
axes[1].set_title('Step 2: Place Random Centroids', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Step 3: Assign points to nearest centroid
kmeans_demo = KMeans(n_clusters=3, init=initial_centroids, n_init=1, max_iter=1, random_state=42)
labels_step3 = kmeans_demo.fit_predict(X_demo)
axes[2].scatter(X_demo[:, 0], X_demo[:, 1], c=labels_step3, s=80, cmap='viridis', alpha=0.6, edgecolors='black')
axes[2].scatter(initial_centroids[:, 0], initial_centroids[:, 1], 
               c='red', s=400, marker='*', edgecolors='black', linewidth=2, label='Centroids')
axes[2].set_title('Step 3: Assign Points to Nearest Centroid', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Step 4: Update centroids and final result
kmeans_final = KMeans(n_clusters=3, random_state=42)
labels_final = kmeans_final.fit_predict(X_demo)
axes[3].scatter(X_demo[:, 0], X_demo[:, 1], c=labels_final, s=80, cmap='viridis', alpha=0.6, edgecolors='black')
axes[3].scatter(kmeans_final.cluster_centers_[:, 0], kmeans_final.cluster_centers_[:, 1],
               c='red', s=400, marker='*', edgecolors='black', linewidth=2, label='Final Centroids')
axes[3].set_title('Step 4: Update Centroids & Repeat Until Convergence', fontsize=14, fontweight='bold')
axes[3].set_xlabel('Feature 1')
axes[3].set_ylabel('Feature 2')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Notes
print("🎯 Notice:")
print("  • Step 3 assignments may vary across runs due to centroid initialization.")
print("  • Different centroid-based algorithms compute cluster centers differently.")
print("  • The algorithm repeats assignment and update steps until centroids stabilize.")

## K-Means Clustering

K-Means is the most widely used clustering algorithm because it's fast, intuitive, and works well when clusters are roughly spherical.

### What Makes K-Means Unique?
K-Means uses the **mean (average position)** of points in a cluster as the centroid. This makes the algorithm computationally efficient and easy to interpret.

### The Meaning of the Mean
The mean is the center point that minimizes squared distance from all other points — much like finding the average location of a group of people standing on a field.

### The K-Means Algorithm (Simplified)
1. Pick the number of clusters (k)
2. Randomly place k centroids
3. Assign points to nearest centroid
4. Recompute centroids as the mean of assigned points
5. Repeat until convergence

### Real-World Applications
- Customer segmentation
- Grouping similar Netflix movies
- Image compression
- Market segmentation

### Strengths
✔ Fast and scalable
✔ Easy to explain
✔ Works well when clusters are compact and separated

### Weaknesses
✘ Must choose k
✘ Sensitive to outliers
✘ Assumes clusters are roughly circular
✘ Struggles with differently shaped clusters

### Example: Clustering Students by Study Habits

In this example, we imagine a school where students differ in two measurable characteristics:
- Hours spent studying per week
- Average course grade

We create synthetic data mimicking three natural groups of students:
- Students who study a lot and receive high grades
- Students who study moderately and perform adequately
- Students who study very little and struggle

K-Means can discover these groups even without labels.

In [ ]:
# (CODE UNCHANGED)
np.random.seed(42)
n_students = 200
X_students, true_labels = make_blobs(n_samples=n_students, centers=3, 
                                     cluster_std=1.0, random_state=42)
X_students[:, 0] = X_students[:, 0] * 5 + 20
X_students[:, 1] = X_students[:, 1] * 10 + 75
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_students)
centers = kmeans.cluster_centers_

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(X_students[:, 0], X_students[:, 1], c='gray', s=80,
           alpha=0.6, edgecolors='black')
plt.xlabel('Study Hours per Week')
plt.ylabel('Average Grade')
plt.title('Before K-Means: All Students Together')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_students[:, 0], X_students[:, 1], c=labels, cmap='viridis',
           s=80, alpha=0.6, edgecolors='black')
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=500, marker='*',
           edgecolors='black', linewidth=2)
plt.xlabel('Study Hours per Week')
plt.ylabel('Average Grade')
plt.title('After K-Means: 3 Distinct Groups Found')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Cluster Profiles:\n")
for i in range(3):
    pts = X_students[labels == i]
    print(f"Cluster {i+1}: {len(pts)} students")
    print(f"  Avg Study Hours: {pts[:,0].mean():.1f}")
    print(f"  Avg Grade: {pts[:,1].mean():.1f}%")
    print()

### Q1: Understanding K-Means
a. What happens if you increase k from 3 to 4? What new type of student group might appear?

b. The cluster centers (red stars) represent the **mean position** of each cluster. In your own words, describe what that means conceptually.

c. Name a scenario where K-Means might perform poorly, and explain why.

### A:
YOUR ANSWER HERE

## Density-Based Clustering: DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) groups points by density instead of distance to a centroid. This gives DBSCAN the ability to find clusters of **arbitrary shapes**, identify **outliers**, and avoid specifying the number of clusters ahead of time.

### Key Insights
- Clusters are formed by areas where points are packed closely.
- Sparse regions form boundaries between clusters.
- Points in very low-density regions get labeled as **noise**.

### Core, Border, and Noise Points
- **Core Point:** Has enough neighbors within radius ε.
- **Border Point:** Near a core point but not dense enough itself.
- **Noise Point:** Belongs to no cluster (too isolated).

### Hyperparameters: eps and min_samples
- **eps:** Max distance for points to be considered neighbors.
- **min_samples:** Minimum neighbors needed to form a dense region (often 5).

If eps is too small → many noise points.
If eps is too large → clusters blend together.

### Why DBSCAN Is Useful
- Finds non-circular clusters
- Works with uneven cluster shapes
- Automatically finds the number of clusters
- Robust to outliers

However, it struggles with highly variable density.

### Example: DBSCAN vs K-Means on Complex Shapes
This example demonstrates that DBSCAN handles irregular shapes (like moon-shaped data) much better than K-Means, which assumes clusters are spherical.

In [ ]:
# (CODE UNCHANGED)
X_moons, _ = make_moons(n_samples=300, noise=0.05, random_state=42)
kmeans_moons = KMeans(n_clusters=2, random_state=42)
labels_kmeans = kmeans_moons.fit_predict(X_moons)

dbscan = DBSCAN(eps=0.2, min_samples=5)
labels_dbscan = dbscan.fit_predict(X_moons)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c='gray', s=80, edgecolors='black', alpha=0.6)
axes[0].set_title('Original Moon-Shaped Data')

axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_kmeans, cmap='viridis', s=80,
               edgecolors='black', alpha=0.6)
axes[1].set_title('K-Means (Incorrect)')

axes[2].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_dbscan, cmap='viridis', s=80,
               edgecolors='black', alpha=0.6)
axes[2].set_title('DBSCAN (Correct)')

plt.tight_layout()
plt.show()

print("DBSCAN automatically detected:")
print(f" - Clusters: {len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)}")
print(f" - Outliers: {(labels_dbscan == -1).sum()}")

### Q2: Understanding DBSCAN
a. Why does DBSCAN work better than K-Means on moon-shaped data?

b. What happens if eps is too small? Too large?

c. Give a real-world dataset where DBSCAN would outperform K-Means.

### A:
YOUR ANSWER HERE

## Connectivity-Based Clustering (Hierarchical Clustering)

Hierarchical clustering builds a **tree-like structure** of clusters, known as a dendrogram. This gives you both a high-level and detailed view of the grouping structure.

Unlike K-Means or DBSCAN, hierarchical clustering shows you:
- How clusters form progressively
- How similar clusters are before merging
- Where natural group boundaries exist

### Two Approaches
#### 1. Agglomerative (Bottom-Up)
- Start with each point as its own cluster
- Gradually merge the closest clusters
- Continue until everything merges into one

#### 2. Divisive (Top-Down)
- Start with one big cluster
- Recursively split into smaller clusters

Agglomerative is more common and easier to compute.

## Agglomerative Clustering

Agglomerative clustering starts with each point on its own and repeatedly merges the closest pair of clusters.

### Linkage Criteria
- **Single linkage:** Minimum distance between clusters
- **Complete linkage:** Maximum distance
- **Average linkage:** Average distance
- **Ward’s method:** Minimizes variance within clusters

Ward’s method is most common since it creates compact, balanced clusters.

## Divisive Clustering (Top-Down)

Divisive methods begin with one large cluster and iteratively split it apart.

### Example: Bisecting K-Means
- Runs K-Means with k=2 to split a cluster
- Repeats splitting until the desired number of clusters is reached

This approach is faster than full hierarchical clustering and gives you a hierarchy without building a full dendrogram.

## Evaluation of Clustering Methods

Since clustering has no labels, evaluation is trickier than supervised learning. We rely on metrics that measure **cohesion** (how tight clusters are) and **separation** (how far apart clusters are).

### 1. Silhouette Score
Measures how similar a point is to its own cluster vs other clusters.
- **Near 1.0:** Great clustering
- **Around 0:** Overlapping clusters
- **Negative:** Wrong cluster assignment

### 2. Rand Index
Measures agreement between your clustering and a known ground truth (if available).
- **1.0:** Identical
- **0.0:** Completely different

In [ ]:
# (CODE UNCHANGED)
np.random.seed(42)
X_eval, _ = make_blobs(n_samples=300, centers=3, cluster_std=0.7, random_state=42)
scores = []

for k in range(2, 7):
    kmeans_eval = KMeans(n_clusters=k, random_state=42)
    labels = kmeans_eval.fit_predict(X_eval)
    score = silhouette_score(X_eval, labels)
    scores.append(score)

plt.figure(figsize=(8, 5))
plt.plot(range(2, 7), scores, marker='o')
plt.title('Silhouette Score vs. K')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True, alpha=0.3)
plt.show()

## Comparison of Methods

Below is a summary of the strengths and weaknesses of the three main clustering families:

| Algorithm | Core Idea | Strengths | Weaknesses | Best Use Cases |
|----------|-----------|-----------|-------------|----------------|
| **K-Means** | Distance to centroids | Fast, simple | Poor on irregular shapes | Circular clusters, large datasets |
| **DBSCAN** | Density of points | Finds arbitrary shapes, detects noise | Sensitive to eps | Geographic clusters, noisy data |
| **Bisecting K-Means** | Recursive splitting | Hierarchical structure | Still spherical clusters | When hierarchy is wanted |

We now visualize how each algorithm handles circular rings.

In [ ]:
# (CODE UNCHANGED)
X_compare, _ = make_circles(n_samples=400, factor=0.5, noise=0.05, random_state=42)
kmeans_compare = KMeans(n_clusters=2, random_state=42)
labels_k = kmeans_compare.fit_predict(X_compare)

dbscan_compare = DBSCAN(eps=0.2, min_samples=5)
labels_d = dbscan_compare.fit_predict(X_compare)

bkm_compare = BisectingKMeans(n_clusters=2, random_state=42)
labels_b = bkm_compare.fit_predict(X_compare)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].scatter(X_compare[:, 0], X_compare[:, 1], c=labels_k, cmap='coolwarm', edgecolor='k')
axes[0].set_title('K-Means')
axes[1].scatter(X_compare[:, 0], X_compare[:, 1], c=labels_d, cmap='coolwarm', edgecolor='k')
axes[1].set_title('DBSCAN')
axes[2].scatter(X_compare[:, 0], X_compare[:, 1], c=labels_b, cmap='coolwarm', edgecolor='k')
axes[2].set_title('Bisecting K-Means')
plt.suptitle('Comparing Clustering Methods')
plt.tight_layout()
plt.show()

## Discussion 💬

Look at the plots above and think about:
1. Which algorithm handles concentric circles correctly? (Hint: DBSCAN)
2. Which struggles to separate inner vs outer rings? (Hint: K-Means)
3. Why does DBSCAN succeed here? (Density-based logic)
4. How would noise affect each algorithm differently?

### Key Takeaway
Different clustering algorithms excel on different types of data. There is **no single best method** — choosing the right approach depends heavily on the structure of your dataset.